In [ ]:
%load_ext autoreload
%autoreload 2

# fetch_player_stats

> Fetch player starts for individual games.

In [ ]:
#| default_exp fetch_player_stats

In [ ]:
#| export
def fetch_player_stats(player_id: str):
    'Fetches player stats for a particular player'
    return f'{player_id}'

In [ ]:
fetch_player_stats("54654")

'54654'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()